In [1]:
import numpy as np
import random
import cv2
import os,sys
from PIL import Image, ImageDraw
import math
from random import shuffle

%matplotlib inline
ROOT_DIR = os.getcwd()

In [2]:
DR_IMAGE_DIR = os.path.join(ROOT_DIR, "GenerateData-GroundTruth/MA/MA_ANN") #DRDataset/SubChallenge_1_Images
    # Load a random image from the images folder
file_names = next(os.walk(DR_IMAGE_DIR))[2]

In [10]:
def randomCoordinates(x = 4000, y = 2800):
    x = random.randint(280,x)  #把黑边空出去了
    y = random.randint(1,y)
    return x, y

def drawCircle(img, x = 800, y = 1000, radius = 50, R = 0, G = 0, B = 255):  # length = 2000, width = 1500
    image = cv2.circle(img,(x,y), radius, (R, G, B), -1)    
    return image

def compareMatrix(res_img, draw_img):
    return (res_img == draw_img).any()

def randomBlackBlock(img, radius, num = 50):
    [x, y, z] = img.shape
    imgZeroCopy = np.zeros([x,y,z], dtype = np.uint8)
    centerlist = []
    for i in range(num):
        #print("i=", i)
        A = 1
        while A:
            m, n = randomCoordinates()
            #print(m, n)
            imgZero = np.zeros([x,y,3], dtype = np.uint8)
            img_draw = drawCircle(imgZero, m, n, radius, R = 1, G = 1, B = 1)
            img_draw = img_draw * img
            #cv2.imwrite('test/test_img_draw%02d.png'%(i), img1)
            #print((img == img_draw).all())
            if (imgZeroCopy == img_draw).all():
                centerlist.append([m, n, 50, 0])
                A = 0
    centerLocation = np.array(centerlist)
    return centerLocation

def calculateLesionCoordinates(img, typeLesion = 1):   #typeLesion 包括1(MA), 2(HR), 3(HE), 4(SE)
    image = Image.fromarray(img)
    img_gray=np.array(image.convert("L")) 
    mask_arr = np.array(img_gray != 0).astype(np.uint8)
    mask_arr = mask_arr*255
    [x,y]=img_gray.shape
    Mask_arr= np.zeros([x,y,3], dtype = np.uint8)

    ROI,contours,hierarchy= cv2.findContours(mask_arr,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    #box = []
    centerlist = [] 
    mylist2 = []
    #for item in data: 
    
    lesionNum = len(contours)
    area = cv2.countNonZero(mask_arr)

    for i in range(len(contours)):
        x,y,w,h=cv2.boundingRect(contours[i]) #（x,y）为矩形左上角的坐标，（w,h）是矩形的宽和高
        centerX = round(x + w/2)
        centerY = round(y + h/2)
        centerlist.append([centerX, centerY, max(math.ceil(w/2), math.ceil(h/2)), typeLesion])
    
    centerLocation = np.array(centerlist)
    #print(centerLocation)
    return centerLocation

In [15]:
newfile_name = file_names[5].strip('_MA.png')
filepath_ma = "GenerateData-GroundTruth/MA/MA_ANN/"+newfile_name+"_MA.png"
filepath_hr = "GenerateData-GroundTruth/HR/HR_ANN/"+newfile_name+"_HR.png"
filepath_he = "GenerateData-GroundTruth/HE/HE_ANN/"+newfile_name+"_HE.png"
filepath_se = "GenerateData-GroundTruth/SE/SE_ANN/"+newfile_name+"_SE.png"
src_ma = cv2.imread(filepath_ma)
#src_ma_gray = cv2.imread(filepath_ma, cv2.COLOR_BGR2RGB)
src_hr = cv2.imread(filepath_hr)
src_he = cv2.imread(filepath_he)
src_se = cv2.imread(filepath_se)
src = src_ma + src_hr + src_he + src_se
#cv2.imwrite('test_src.png', src)
(x,y,z) = src.shape
BlackCenterLocation = randomBlackBlock(src, 50)
LesionCenterLocation = calculateLesionCoordinates(src_ma, 1)
CenterLocation = np.concatenate((BlackCenterLocation, LesionCenterLocation), axis = 0)
print(len(CenterLocation))
print(CenterLocation)
np.random.shuffle(CenterLocation)
print(len(CenterLocation))
print(CenterLocation)
# img1 = np.ones((x,y,z))*0
# img_draw = drawCircle(img1, x = 1056, y = 700, radius = 50, R = 1, G = 1, B = 1)
# #img_draw = drawCircle(img1, x = 60, y = 60, radius = 50, R = 1, G = 1, B = 1)
# img_draw = img_draw * src
# cv2.imwrite('test_img_draw.png', img_draw)
# cv2.imwrite('test_img_draw1.png', img1)
# print((img1 == img_draw).all())

79
[[1772 1184   50    0]
 [ 646  670   50    0]
 [3921 1131   50    0]
 [3906 2436   50    0]
 [ 692 1947   50    0]
 [ 735  638   50    0]
 [3786 1639   50    0]
 [3871 2656   50    0]
 [3160 1204   50    0]
 [3032   10   50    0]
 [ 375 1955   50    0]
 [3585 1041   50    0]
 [ 293 1099   50    0]
 [3349 2409   50    0]
 [3801 1065   50    0]
 [1045 1226   50    0]
 [2617 2306   50    0]
 [1909  396   50    0]
 [3077 2588   50    0]
 [2647 1053   50    0]
 [3246  141   50    0]
 [3829 2389   50    0]
 [3934 1678   50    0]
 [2373  921   50    0]
 [1781 2187   50    0]
 [3137 1690   50    0]
 [1735 2311   50    0]
 [1534  575   50    0]
 [3733 1709   50    0]
 [1335 1398   50    0]
 [2608 2235   50    0]
 [2652 1292   50    0]
 [ 755  946   50    0]
 [2787  858   50    0]
 [3005   93   50    0]
 [ 427 2177   50    0]
 [2293  337   50    0]
 [1121 2250   50    0]
 [1350 1945   50    0]
 [2604  896   50    0]
 [1054 2588   50    0]
 [1085 1967   50    0]
 [3568 1360   50    0]
 [1074 1

In [12]:
shuffle(CenterLocation)